In [1]:
# Mount Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Specify working directory.
FOLDERNAME = 'cs231n/project/'
%cd /content/drive/My\ Drive/$FOLDERNAME

# Ensure that the Python interpreter of the Colab VM can load python files from
# within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive
/content/drive/My Drive/cs231n/project


## Load dataset and labels

In [2]:
import numpy as np
import os

base_dir = '/content/drive/My Drive/cs231n/project/dataset/shaped_cropped_data'

train_data = np.load(os.path.join(base_dir, 'train_data_p12.npy'))
train_labels = np.load(os.path.join(base_dir, 'train_labels_p12.npy'))
train_actions = np.load(os.path.join(base_dir, 'train_actions_p12.npy'), allow_pickle=True)

#val_labels = np.load(os.path.join(base_dir, 'val_labels_10.npy'))
#val_data = np.load(os.path.join(base_dir, 'val_data_10.npy'))
#val_actions = np.load(os.path.join(base_dir, 'val_actions_10.npy'), allow_pickle=True)

In [3]:
print('Training data shape: ', train_data.shape)
print('Training label shape: ', train_labels.shape)
print('Training action shape: ', train_actions.shape)
print('Train Label=1 ratio:', np.sum(train_labels) /train_labels.shape[0])
#print('Validation data shape: ', val_data.shape)
#print('Validation label shape: ', val_labels.shape)
#print('Validation action shape: ', val_actions.shape)
#print('Validation Label=1 ratio:', np.sum(val_labels) / val_labels.shape[0])

Training data shape:  (1007, 12, 224, 224, 3)
Training label shape:  (1007,)
Training action shape:  (1007,)
Train Label=1 ratio: 0.24925521350546176


In [4]:
mapping = {
    'CALL': 0,
    'COUG': 1,
    'DRIN': 2,
    'SCRA': 3,
    'SNEE': 4,
    'STRE': 5,
    'WAVE': 6,
    'WIPE': 7,
}
train_action_labels = [mapping[x] for x in train_actions]
#val_action_labels = [mapping[x] for x in val_actions]

# One-hot encoding.
train_action_labels = np.eye(8)[train_action_labels] 
#val_action_labels = np.eye(8)[val_action_labels] 

## Model 1: CNN and LSTM (basic architecture)

In [5]:
from keras.layers import Input, TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras import models, Model
from keras.applications import vgg16

frames, channels, rows, columns = 12, 3, 224, 224

video = Input(shape=(frames,rows,columns,channels))
cnn_base = vgg16.VGG16(input_shape=(rows,columns,channels),
                 weights="imagenet",
                 include_top=False)
                 #include_top=True) # want the final dense layers
cnn_base.trainable = False

#cnn = Model(cnn_base.input, cnn_base.layers[-3].output) # -3 is the 4096 layer
cnn_out = GlobalAveragePooling2D()(cnn_base.output)
cnn = Model(cnn_base.input, cnn_out)
encoded_frames = TimeDistributed(cnn)(video)
encoded_sequence = LSTM(32)(encoded_frames)
x = Dense(64, activation="relu")(encoded_sequence)
x = Dense(32, activation="relu")(x)
x = Dropout(0.2)(x)
outputs = Dense(8, activation="softmax")(x) # n_labels = 8

model = Model(video, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 12, 224, 224, 3)] 0         
_________________________________________________________________
time_distributed (TimeDistri (None, 12, 512)           14714688  
_________________________________________________________________
lstm (LSTM)                  (None, 32)                69760     
_________________________________________________________________
dense (Dense)                (None, 64)                2112      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 264 

In [6]:
import keras
callbacks_list = [keras.callbacks.EarlyStopping(monitor='accuracy', patience=5),
                  keras.callbacks.ModelCheckpoint(
                      filepath='cnn_lstm_model_v1',
                      monitor='val_loss',
                      save_best_only=True
                  ),
                  keras.callbacks.ReduceLROnPlateau(
                      monitor="val_loss", factor=0.1, patience=3
                  )]

from keras import optimizers

optimizer_new=optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer_new,
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

In [7]:
history = model.fit(train_data, train_action_labels,
                    #validation_data=(val_data, val_action_labels),
                    batch_size=10, epochs=10,
                    callbacks=callbacks_list
                    )

Epoch 1/10
101/101 [==============================] - 56s 277ms/step - loss: 2.0631 - accuracy: 0.1586
Epoch 2/10
101/101 [==============================] - 23s 225ms/step - loss: 2.0157 - accuracy: 0.1634
Epoch 3/10
101/101 [==============================] - 23s 224ms/step - loss: 1.9951 - accuracy: 0.2082
Epoch 4/10
101/101 [==============================] - 23s 224ms/step - loss: 1.9901 - accuracy: 0.1739
Epoch 5/10
101/101 [==============================] - 23s 224ms/step - loss: 1.9839 - accuracy: 0.2318
Epoch 6/10
101/101 [==============================] - 23s 224ms/step - loss: 1.9863 - accuracy: 0.2135
Epoch 7/10
101/101 [==============================] - 23s 224ms/step - loss: 1.9916 - accuracy: 0.1988
Epoch 8/10
101/101 [==============================] - 23s 224ms/step - loss: 1.9852 - accuracy: 0.2213
Epoch 9/10
101/101 [==============================] - 23s 224ms/step - loss: 1.9811 - accuracy: 0.2060
Epoch 10/10
101/101 [==============================] - 23s 224ms/step - l

In [ ]:
acc=history.history["acc"]
val_acc=history.history["val_acc"]
loss=history.history["loss"]
val_loss=history.history["val_loss"]
epoch_stop = 10
epochs=np.arange(1, epoch_stop+1)

plt.plot(epochs, acc, label = "Train", color = 'darkblue')
plt.plot(epochs, val_acc, label = "Valid", linestyle="--", color = 'darkred')
plt.legend()
plt.title('Train/Valid Accuracy')
plt.show()

plt.plot(epochs, loss, label = "Train", color = 'darkblue')
plt.plot(epochs, val_loss, label = "Valid", linestyle="--", color = 'darkred')
plt.legend()
plt.title('Train/Valid Loss')
plt.show()

NameError: ignored

## Evaluate model on test dataset

In [ ]:
base_dir = '/content/drive/My Drive/cs231n/project/dataset/shaped_data'

test_data = np.load(os.path.join(base_dir, 'test_data.npy'))
text_labels = np.load(os.path.join(base_dir, 'test_labels.npy'))

In [ ]:
# Evaluate on test dataset.
model = load_model('cnn_lstm_model_v1')
model.evaluate(test_data, test_labels)

In [ ]:
# preds = model.predict_classes(X_test)
# print(classification_report(train_labels, preds))